### Maximum Bipartite Matching

We will implement a maximum bipartite matching algorithm using the method of `augmenting paths`. Let `V` and `U` be the disjoint sets of vertices of our bipartite graph. Then we start with an initial matching which is the empty set and iteratively augment this matching. Each iteration involves finding an augmenting path connecting a pair of `free vertices`, one from `V` and the other from `U`, then adding and removing alternating edges on this path into the current matching. The algorithm ends when an augmenting path can no longer be found and the current matching is the maximum. 

We can implement this iterative algorithm efficiently by using bredth-first search to find augmenting paths. The pseudocode is shown below (borrowed from Levitin textbook):

<img src="pseudocode.png" width="500" height="550">


In [69]:
def matching_add(U_mate, edge, V_free, U_free):
    v, u = edge
    U_mate[u] = v
    V_free[v] = False
    U_free[u] = False  

def matching_remove(U_mate, edge, V_free, U_free):
    v, u = edge
    U_mate[u] = 0
    V_free[v] = True
    U_free[v] = True

def reset_Q(V_free):
    return [v for v in V_free if V_free[v]]    

def max_bipartitie_matching(adjacency_list, V, U, verbose=False):
    # initialize the matching
    U_mate = {i:0 for i in list(U)} # mates of U vertices set to zero, i.e. not matched
    # free vertices
    V_free = {i:True for i in V}
    U_free = {i:True for i in U}
    # initialize the BFS queue with all free vertices from V
    Q = list(V)
    # initialize labels
    labels = {i:0 for i in range(1, len(adjacency_list)+1)}
    
    # start BFS
    while len(Q)>0:
        if verbose: print(f"Q: {Q}")
        w = Q.pop(0)
        if verbose: print(f"w: {w}, V_free: {V_free}, U_free: {U_free}, Q: {Q}, labels: {labels}")
        if w in V:
            for u in adjacency_list[w]:
                if verbose: print(f"u: {u}")
                if U_free[u]:
                    # found an augmenting path, start augmentation procedure
                    if verbose: print("u is free, found augmenting path!")
                    matching_add(U_mate, (w,u), V_free, U_free)
                    v = w
                    while labels[v] != 0:
                        u = labels[v]
                        matching_remove(U_mate, (v,u), V_free, U_free)
                        v = labels[u]
                        matching_add(U_mate, (v,u), V_free, U_free)
                    # after augmentation, reset labels and BFS queue, exit the loop
                    labels = {i:0 for i in range(1, len(adjacency_list)+1)}    
                    Q = reset_Q(V_free)
                    break
                else:
                    # u is already matched, label if not already labelled
                    if verbose: print(f"u already matched")
                    if U_mate[u]!=w and labels[u] == 0:
                        if verbose: print("labelling u with w")
                        labels[u] = w
                        Q.append(u)    
        else:
            # label mate v of w with w and add it to BFS queue
            v = U_mate[w]
            if verbose: print(f"mate v of w: {labels[w]}")
            labels[v] = w
            Q.append(v)
            if verbose: print(f"Labelled v with w: {labels[v]}")

    # return the maximum matching
    M = [(U_mate[u],u) for u in U_mate if U_mate[u]!=0]

    return M

In [65]:
# define an example (undirected) bipartite graph
adjacency_list = {1:[6,7], 2:[6], 3:[6,8], 4:[8,9,10], 5:[9,10], 6:[1,2,3], 7:[1], 8:[3,4], 9:[4,5], 10:[4,5]}
V = {1,2,3,4,5}
U = {6,7,8,9,10}

In [70]:
max_M = max_bipartitie_matching(adjacency_list, V, U)
print(f"Maximum matching: {max_M}")

Maximum matching: [(2, 6), (1, 7), (3, 8), (4, 9), (5, 10)]
